### A

In [4]:
n, m, p = map(int, input().split())

if n < m:
    print(0)
elif n == m:
    print(1)
else:
    r = (n - m) // p + 1
    
    print(r)

200000 314 318
628


### B

In [9]:
n = int(input())

result = [[False] * 101 for _ in range(101)]
for _ in range(n):
    a, b, c, d = map(int, input().split())
    
    for i in range(a, b):
        for j in range(c, d):
            result[i][j] = True

sv = 0
for i in range(101):
    for j in range(101):
        sv += result[i][j]
        
print(sv)

3
0 1 0 1
0 3 0 5
5 10 0 10
65


### C

In [12]:
n, d, p = map(int, input().split())
li = sorted(map(int, input().split()))

result = sum(li)

while li:
    is_needed = False
    temp_sv = 0
    for _ in range(d):
        if li:
            temp_sv += li.pop()
            
    if p < temp_sv:
        saved = temp_sv - p
        result -= saved
        is_needed = True
        
    if not is_needed:
        break
        
print(result)

8 3 1000000000
1000000000 1000000000 1000000000 1000000000 1000000000 1000000000 1000000000 1000000000
3000000000


### D

In [12]:
def recur(res, sv):
    global max_sv
    
    last = res.pop()
    for v in res:
        nres = [w for w in res if w != v]
        
        nsv = sv + values[v][last]
        
        if len(nres) <= 1:
            max_sv = max(max_sv, nsv)
        
        if nres:
            recur(nres, nsv)
            
    res.append(last)
    
n = int(input())
li = [list(map(int, input().split())) for _ in range(n - 1)]
    
values = [[0] * n for _ in range(n)]
for i in range(n - 1):
    for j in range(n - i - 1):
        values[i][i + j + 1] = li[i][j]
        
max_sv = 0

res = list(range(n))
recur(res, 0)
res = list(range(n - 1))
if len(res) >= 2:
    recur(res, 0)

print(max_sv)

4
1 5 4
7 8
6
13


In [15]:
# https://github.com/tatyam-prime/SortedSet/blob/main/SortedSet.py
import math
from bisect import bisect_left, bisect_right
from typing import Generic, Iterable, Iterator, TypeVar, Union, List
T = TypeVar('T')

class SortedSet(Generic[T]):
    BUCKET_RATIO = 50
    REBUILD_RATIO = 170

    def _build(self, a=None) -> None:
        "Evenly divide `a` into buckets."
        if a is None: a = list(self)
        size = self.size = len(a)
        bucket_size = int(math.ceil(math.sqrt(size / self.BUCKET_RATIO)))
        self.a = [a[size * i // bucket_size : size * (i + 1) // bucket_size] for i in range(bucket_size)]
    
    def __init__(self, a: Iterable[T] = []) -> None:
        "Make a new SortedSet from iterable. / O(N) if sorted and unique / O(N log N)"
        a = list(a)
        if not all(a[i] < a[i + 1] for i in range(len(a) - 1)):
            a = sorted(set(a))
        self._build(a)

    def __iter__(self) -> Iterator[T]:
        for i in self.a:
            for j in i: yield j

    def __reversed__(self) -> Iterator[T]:
        for i in reversed(self.a):
            for j in reversed(i): yield j
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return "SortedSet" + str(self.a)
    
    def __str__(self) -> str:
        s = str(list(self))
        return "{" + s[1 : len(s) - 1] + "}"

    def _find_bucket(self, x: T) -> List[T]:
        "Find the bucket which should contain x. self must not be empty."
        for a in self.a:
            if x <= a[-1]: return a
        return a

    def __contains__(self, x: T) -> bool:
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        return i != len(a) and a[i] == x

    def add(self, x: T) -> bool:
        "Add an element and return True if added. / O(√N)"
        if self.size == 0:
            self.a = [[x]]
            self.size = 1
            return True
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i != len(a) and a[i] == x: return False
        a.insert(i, x)
        self.size += 1
        if len(a) > len(self.a) * self.REBUILD_RATIO:
            self._build()
        return True

    def discard(self, x: T) -> bool:
        "Remove an element and return True if removed. / O(√N)"
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i == len(a) or a[i] != x: return False
        a.pop(i)
        self.size -= 1
        if len(a) == 0: self._build()
        return True
    
    def lt(self, x: T) -> Union[T, None]:
        "Find the largest element < x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] < x:
                return a[bisect_left(a, x) - 1]

    def le(self, x: T) -> Union[T, None]:
        "Find the largest element <= x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] <= x:
                return a[bisect_right(a, x) - 1]

    def gt(self, x: T) -> Union[T, None]:
        "Find the smallest element > x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] > x:
                return a[bisect_right(a, x)]

    def ge(self, x: T) -> Union[T, None]:
        "Find the smallest element >= x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] >= x:
                return a[bisect_left(a, x)]
    
    def __getitem__(self, x: int) -> T:
        "Return the x-th element, or IndexError if it doesn't exist."
        if x < 0: x += self.size
        if x < 0: raise IndexError
        for a in self.a:
            if x < len(a): return a[x]
            x -= len(a)
        raise IndexError
    
    def index(self, x: T) -> int:
        "Count the number of elements < x."
        ans = 0
        for a in self.a:
            if a[-1] >= x:
                return ans + bisect_left(a, x)
            ans += len(a)
        return ans

    def index_right(self, x: T) -> int:
        "Count the number of elements <= x."
        ans = 0
        for a in self.a:
            if a[-1] > x:
                return ans + bisect_right(a, x)
            ans += len(a)
        return 
    
def recur(res, sv):
    global max_sv
    
    last = res[-1]
    res.discard(last)
    
    for i in range(len(res)):
        fv = res[i]
        nsv = sv + values[fv][last]
        
        if len(res) <= 1:
            max_sv = max(max_sv, nsv)
        else:
            res.discard(fv)
            recur(res, nsv)
            res.add(fv)
            
    res.add(last)
    
n = int(input())
li = [list(map(int, input().split())) for _ in range(n - 1)]
    
values = [[0] * n for _ in range(n)]
for i in range(n - 1):
    for j in range(n - i - 1):
        values[i][i + j + 1] = li[i][j]
        
max_sv = 0

if n % 2 == 0:
    temp_res = list(range(n))
    res = SortedSet()
    for v in temp_res:
        res.add(v)
        
    recur(res, 0)
else:
    for to_except in range(n):
        temp_res = [v for v in range(n) if v != to_except]
        res = SortedSet()
        for v in temp_res:
            res.add(v)        
        
        recur(res, 0)

print(max_sv)

3
1 2
3
3


### E

In [10]:
n = int(input())
li = list(map(int, input().split()))

nli = [[] for _ in range(n + 1)]
for i in range(n):
    v = li[i]
    
    nli[v].append(i)
    
result = 0
for num in range(1, n + 1):
    diffs = []
    for i in range(len(nli[num]) - 1):
        diffs.append(nli[num][i + 1] - nli[num][i] - 1)
        
    total = 0
    
    cur_sum = 0
    cur = 0
    for v in diffs:
        cur += v
        cur_sum += cur
        
        
    total = cur_sum
    cur_w = len(diffs)
    for i in range(len(diffs) - 1):
        cur_sum -= diffs[i] * cur_w
        total += cur_sum
        cur_w -= 1
        
    result += total
    
print(result)

13
9 7 11 7 3 8 1 13 11 11 11 6 13
20


In [13]:
8*7*6*5*4*3*2*1

40320

In [20]:
import io, os, sys
sys.setrecursionlimit(200010)
# input = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

def dfs(value):
    global route
    stack.append(value)
    in_stack[value] = True
    
    if value == target:
        route = stack.copy()
    
    if direction == 1:
        for v in li[value]:
            if not in_stack[v] and v != to_evade:
                dfs(v)
    else:
        for i in range(len(li[value]) - 1, -1, -1):
            v = li[value][i]
            
            if not in_stack[v] and v != to_evade:
                dfs(v)
                
    stack.pop()
    in_stack[value] = False
    
n, m = map(int, input().split())
a, b, c = map(int, input().split())

li = [[] for _ in range(n + 1)]
for _ in range(m):
    u, v = map(int, input().split())
    li[u].append(v)
    li[v].append(u)
    
for i in range(n + 1):
    li[i].sort()
    
direction = 1
target = a
to_evade = c
route = []
in_stack = [False] * (n + 1)
stack = []
dfs(b)

if not route:
    print('No')
else:
    st1 = set(route)
    st1.discard(b)
    st1.discard(a)

    direction = 0
    target = c
    to_evade = a
    route = []
    in_stack = [False] * (n + 1)
    stack = []
    dfs(b)
    
    if not route:
        print('No')
    else:
        st2 = set(route)
        st2.discard(c)
        st2.discard(a)
        
        if not (st1 & st2):
            print('Yes')
        else:
            direction = 0
            target = a
            to_evade = c
            route = []
            in_stack = [False] * (n + 1)
            stack = []
            dfs(b) 
            
            st1 = set(route)
            st1.discard(b)
            st1.discard(a)
            
            direction = 1
            target = c
            to_evade = a
            route = []
            in_stack = [False] * (n + 1)
            stack = []
            dfs(b)
            
            st2 = set(route)
            st2.discard(c)
            st2.discard(a)
            
            if not (st1 & st2):
                print('Yes')
            else:
                print('No')

Yes


In [16]:
{2, 3} & {3, 6}

{3}